In [1]:
using Pkg;
Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl#verbose")


     Cloning git-repo `https://github.com/FerreolS/OptimPackNextGen.jl#verbose`
    Updating git-repo `https://github.com/FerreolS/OptimPackNextGen.jl#verbose`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
    Updating `~/.julia/environments/v1.6/Project.toml`
  [080ffe24] ~ OptimPackNextGen v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose` ⇒ v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose#master`
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [080ffe24] ~ OptimPackNextGen v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose` ⇒ v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose#master`


In [2]:
include("../src/SphereIFSCalib.jl")
using TwoDimensional, Zygote,StatsBase, Plots, OptimPackNextGen
plotly()

┌ Info: Precompiling OptimPackNextGen [080ffe24-4a27-4ab0-ad71-7b624e73ef8d]
└ @ Base loading.jl:1317


Plots.PlotlyBackend()

In [3]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference


In [5]:
# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=50);
a0= rand(Float64,3);
fwhm0= rand(Float64,3)*10;
laser =  LaserModel(λlaser,a0,fwhm0)
lmod = LensletModel(λ0,laser.nλ-1,bbox);



In [7]:

Cinit = [[ 6.2 10 0 ]; [20 100 0 ]]
UpdateDispModel(lmod.dmodel,Cinit)
limage = LensletLaserImage(lmod,laser)
heatmap(limage)


<!DOCTYPE html>
 
 
 Plots.jl

In [9]:
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end

data= limage ;
cost = likelihood(a0,fwhm0,Cinit)


0.0

In [10]:
@time ∇cost = gradient(likelihood,a0,fwhm0,Cinit)


 30.447018 seconds (59.65 M allocations: 3.407 GiB, 5.07% gc time, 99.92% compilation time)


([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0 0.0 0.0; 0.0 0.0 0.0])

In [12]:

@show  cost = likelihood(a0,fwhm0.+ 2.,Cinit)
@time  ∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,Cinit)


cost = likelihood(a0, fwhm0 .+ 2.0, Cinit) = 4.522673602624989
  0.001409 seconds (7.79 k allocations: 414.078 KiB)


([18.465588156023802, 23.184033798203394, 3.814221737183159], [1.5651155144533957, 2.7329429386148942, 0.06396756504871269], [-1.4299638705986246 -0.12103226560884571 -0.014691947913076637; 0.11368534276387288 -0.11192515649378973 -0.013663940068454748])

In [13]:

function  likelihood(x::Vector{Float64}) 
    (a,fwhm,c) = (x[1:3],x[4:6],reshape(x[7:12],2,:));
    UpdateDispModel(lmod.dmodel, c);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end


likelihood (generic function with 2 methods)

In [15]:
x0 = vcat([a0[:],fwhm0[:],Cinit[:] ]...)
cost = likelihood(x0)
@time ∇cost = gradient(likelihood,x0)

@time  ∇cost = gradient(likelihood,x0)


  0.001454 seconds (7.82 k allocations: 416.578 KiB)
  0.001219 seconds (7.82 k allocations: 416.578 KiB)


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],)

In [25]:

xtrue = vcat([a0[:],fwhm0[:],Cinit[:]]...)
xinit = xtrue  .+ 0.1 .* rand(Float64,12)
@time  ∇cost = gradient(likelihood,xinit )

@time xopt = vmlmb(likelihood, xinit; verb=true)

  0.001268 seconds (7.82 k allocations: 416.578 KiB)
# ITER   EVAL   REJECTS          F(X)           ||G(X)||    STEP
#-----------------------------------------------------------------
     0      1      0    4.3386623468246809E-02  2.54E+00  0.00E+00
     1      3      0    4.1650348488224322E-03  4.15E-01  1.22E-02
     2      4      0    3.0881553607110005E-03  8.57E-02  1.00E+00
     3      6      0    2.7063058877714796E-03  1.33E-01  5.00E+00
     4      7      0    1.8798181989083224E-03  1.91E-01  1.00E+00
     5      8      0    1.1451994800762014E-03  1.15E-01  1.00E+00
     6      9      0    1.0478316635577373E-03  1.39E-01  1.00E+00
     7     10      0    9.0433199453143504E-04  5.56E-02  1.00E+00
     8     11      0    8.2715527133468301E-04  5.97E-02  1.00E+00
     9     12      0    5.9545767490877977E-04  1.05E-01  1.00E+00
    10     13      0    3.0173508875422112E-04  1.25E-01  1.00E+00
    11     15      0    2.4717942825515214E-04  1.23E-01  2.91E-01
    12     

12-element Vector{Float64}:
  0.5941278846615855
  0.927610754925392
  0.0758554233269061
  8.517761289190174
  9.50555877578142
  8.15701882323592
  6.200062127736679
 20.000043227787256
  9.989425055340085
 99.99383530356037
  0.08212145524383685
  0.041804861638604585

In [26]:
(aopt,fwhmopt,copt) = (xopt[1:3],xopt[4:6],reshape(xopt[7:12],2,:))
(ainit,fwhminit,cinit) = (xinit[1:3],xinit[4:6],reshape(xinit[7:12],2,:))
(ainit,aopt,a0)

([0.6187874183228615, 0.931663760112129, 0.08269023937990228], [0.5941278846615855, 0.927610754925392, 0.0758554233269061], [0.5941345760799563, 0.9276074033171382, 0.07585185125114302])